# Pandas bevezető

Ez a bevezető az Üzleti Intelligencia tárgy laborjához készült. A pandas könyvtárhoz kívánok egy rövid bevezetőt nyújtani. Az anyag folyamatosan bővül.

Angol nyelven kiváló erőforrásokat találhatsz:

1. A hivatalos dokumentációhoz tartozó rövid bevezető: http://pandas.pydata.org/pandas-docs/stable/10min.html
2. Brandon Rhodes tutorialja a 2015-ös Pyconon (kicsit hosszabb, de megéri): https://www.youtube.com/watch?v=5JnMutdy6Fw

# Mi az a pandas?

A pandas egy adatkezelő függvénykönyvtár, amit a Data Scientistek nagyon szeretnek. Az adatokat táblázatos formában kezeli. A táblázatoknak van fejlécük és indexük. Heterogén adatokat tárolhatunk: szöveges, számadat, igazságérték, dátum stb. Az adatok típusa egy oszlopon belül sem kell, hogy azonos legyen.

A pandas remekül együttműködik a Python gépi tanuló könyvtárával (scikit-learn) és a legelterjedtebb vizualizációs könyvtárakkal (matplotlib, seaborn).

In [ ]:
import pandas as pd  # konvenció szerint pd aliast használunk
%matplotlib inline
import matplotlib
import numpy as np

# tegyük szebbé a grafikonokat
matplotlib.style.use('ggplot')
matplotlib.pyplot.rcParams['figure.figsize'] = (15, 3)
matplotlib.pyplot.rcParams['font.family'] = 'sans-serif'

In [ ]:
grades = pd.DataFrame(
    {
        'subject': ['analízis 1', 'digitális technika 1', 
                    'fizika 1', 'mikmak', 'programozás alapjai 1', 'szoftvertechonológia',
                   'bevezetés a számításelméletbe 1'],
        'grade': [3, 4, 3, 2, 5, 1, 4],
        'teacher': ['a', 'b', 'a', 'c', 'd', 'd', 'd'],
        'semester': [1, 1, 2, 2, 1, 3, 1],
    }
)
grades

A DataFrame elejét a .head függvénnyel, a végét a .tail-lel nézhetjük meg. Ennek akkor van jelentősége, ha nagy táblázatokkal dolgozunk.

In [ ]:
grades.head()

Alapértelmezetten 5 sort ír ki, de megadhatjuk paraméterként pontosan hány sort szeretnénk látni.

In [ ]:
grades.tail(2)

# Egyszerű navigálás a DataFrame-ben

Fontos megemlíteni, hogy minden művelet egy új DataFrame-mel tér vissza, beleértve a `head` és a `tail` függvényeket is, azonban ezek az új DataFrame-ek nem tényleges másolatok, hanem csak ún. slice-ok az eredetiből. A `copy` függénnyel tudunk másolatot készíteni.

A táblázat első oszlopa kitüntetett, ez a DataFrame indexe, ezzel tudjuk azonosítani a sorokat. Ugyan nem követelmény, hogy unikus legyen, de praktikus unikusra választani. Egynél több oszlopot is használhatunk indexként (multiindex).

## Oszlopok kiválasztása

Lekérhetünk csak bizonyos oszlopokat, ekkor egy új DataFrame-et kapunk, ami a választott oszlopokat tartalmazza csak. Egy oszlopot a szögletes zárójelekkel tudunk indexelni.

In [ ]:
grades['teacher']

Amennyiben az oszlop neve nem tartalmaz szóközöket, attribútumként is elérjük.

In [ ]:
grades.teacher

A kapott eredmény nem tűnik táblázatnak és valóban más típusú

In [ ]:
type(grades.teacher)

Amikor a dimenziók száma egyre csökken, `Series` objektumot kapunk vissza.

Egynél több oszlop kiválasztásánál dupla zárójelet kell használni.

In [ ]:
grades[['grade', 'teacher']]

Sorszámmal is indexelhetjük az oszlopokat.

In [ ]:
grades[[0, 3]]

In [ ]:
grades[list(range(3))]  # 0,1,2-es oszlopok

## Sorok kiválasztása

A sorokat többféleképpen indexelhetjük:
1. index szerint
2. sorszám szerint

Mielőtt index szerint kérdeznénk le egy sort, állítsuk át az indexet valami beszédesebbre.


In [ ]:
grades = grades.set_index('subject')
grades

In [ ]:
grades.loc['fizika 1']  # a loc után [] kell!

Mivel egyetlen sort kértünk, megint `Series` objektumot kapunk.

In [ ]:
type(grades.loc['fizika 1'])

A Series egy egydimenziós adatsor, gondolhatunk rá úgy, mint a DataFrame egy sorára.

## Indexelés sor szerint

In [ ]:
grades.iloc[1:3]  # utolsó index nincs benne, [1, 3)

# Szűrés

A DataFrame-ek sorain egyszerre végezhetünk műveleteket:

In [ ]:
grades.semester == 1

Ekkor azokra a sorokra kapunk True-t, ahol igaz a feltétel. A feltételek kombinálhatóak, a zárójelezésre figyelni kell:

In [ ]:
(grades.semester == 1) & (grades.teacher == 'a')

Az igazságértékek sorozatával pedig szűrhetjük a DataFrame-eket:

In [ ]:
grades[grades.semester==1]

In [ ]:
grades[(grades.semester == 1) & (grades.teacher == 'a')]

# Vektoros műveletvégzés

A pandas DataFrame-jeire és Series-eire az aritmetikai operátorok túl vannak terhelve, ezért egyszerre végezhetünk az egész táblán műveleteket.

In [ ]:
grades[['grade', 'semester']] + 15

## Az index is hasonlóan manipulálható 

In [ ]:
grades.index.str.upper()

Át is állítható:

In [ ]:
grades.index = grades.index.str.upper()
grades

Majd visszaállítható:

In [ ]:
grades.index = grades.index.str.lower()
grades

# Több DataFrame kezelése, merge

In [ ]:
credits = pd.DataFrame(
    {
        'subject': ['analízis 1', 'fizika 1', 'programozás alapjai 2'],
        'credit': [7, 5, 5]
    }
)
credits

## Hány kreditet érnek a tárgyak, amikre jegyet kaptunk?

In [ ]:
d = grades.merge(credits, left_index=True, right_on='subject', how='outer')

d

### A paraméterek magyarázata:

1. `left_index`: a baloldali DataFrame (grades ebben az esetben) indexe mentén joinoljon.
2. `right_on`: a jobboldali DataFrame (credits) subject mezője mentén joinoljon.
3. `how`: inner/outer. Egyezik az SQL-es joinnal.

### A joinolni kívánt mező kétféle lehet:

1. index: ekkor az indexként használt oszlopot próbálja a másik táblázattal joinolni. left_index=True-ra kell állítani.
2. nem index: egy vagy több nem indexként használt oszlop mentén próbál meg joinolni. left_on=col1 vagy left_on=[col1, col2]

In [ ]:
grades.merge(credits, left_index=True, right_on='subject', how='inner')

## Érvénytelen adatok eldobása 

Látható, hogy nem minden tárgyhoz sikerült megtalálni a kreditszámot, hiszen nem mindegyik szerepelt a credits táblában.

A pandas NaN (not a number) tölti fel a hiányzó mezőket. Szerencsére a legtöbb műveletnek megmondhatjuk, hogy hogyan bánjon a NaN-okkal. Meg is szabadulhatunk tőlük:

In [ ]:
d = d.dropna()
d

# Szélsőérték-keresés (max, idxmax, argmax)

Sokszor vagyunk kíváncsiak arra, hogy mi egy oszlop maximuma/minimuma vagy éppen hol veszi fel ezt az értéket (argmax). A pandas tartalmaz erre beépített függvényeket.

## DataFrame-re

Melyik tárgyból kaptuk a legjobb jegyet?

In [ ]:
print(type(grades.max()))
grades.max()

A `max` függvény egy Series-zel tér vissza, ami minden oszlop maximumát tartalmazza. Sorokra is felthetjük ugyanezt a kérdést, bár erre az adatra nem sok értelme van:

In [ ]:
grades.max(axis=1)

Hol veszi fel a maximumát?

In [ ]:
# grades.idxmax()  # hibát kapunk, mert az egyik oszlop szöveges
grades[['grade', 'semester']].idxmax()

Indexelhetünk is a visszakapott értékekkel, így a legmagasabb értékeket tartalmazó sorokat kapjuk meg.

In [ ]:
grades.loc[grades[['grade', 'semester']].idxmax()]

A számadatot tartalmazó oszlopok közül megkaptuk, hogy melyik sornál veszi fel a maximális értéket. Holtverseny esetén a legelső előfordulást adja vissza.

## Series-re

Series esetén egyszerűbb a dolgunk, a `max` és az `argmax` függvényeket használhatjuk.

In [ ]:
grades.grade.max(), grades.grade.argmax()

# Csoportosítás (groupby)

A groupby függvénnyel tetszőleges oszlop mentén csoportosíthatjuk a DataFrame-et.

In [ ]:
g = credits.groupby('credit')

In [ ]:
g.groups

Nem csak oszlop szerint tudunk csoportosítani, hanem tetszőleges kifejezés szerint.

In [ ]:
credits.credit % 3

Eszerint groupby:

In [ ]:
credits.groupby(credits.credit % 3)

Csoportonként végezhetünk műveleteket:

In [ ]:
grades.groupby("semester").mean()

Egynél több oszlop szerint is csoportosíthatunk, ekkor az olszopok értékeinek összes kombinációja (direkt szorzat) szerepelni fog az indexben.

A `size` függvénnyel az egyes csoportok elemeinek számát kérhetjük le.

In [ ]:
grades.groupby(["semester", "teacher"]).size()

Ismét `Series` objektumot kaphatunk. A könnyebb olvashatóság kedvéért `DataFrame`-é konvertálhatjuk:

In [ ]:
grades.groupby(["semester", "teacher"]).size().to_frame()

# Rendezés

A DataFrame-eket többféleképpen rendezhetjük.

Index szerint rendezve:

In [ ]:
grades.sort_index()

Illetve egy vagy több oszlop szerint rendezve:

In [ ]:
grades.sort_values(['grade', 'semester'])

Fordított sorrendben:

In [ ]:
grades.sort_index(ascending=False)

# Vizualizáció (plot)

A matplotlib modullal együttműködve rengeteg vizualizációs lehetőségünk van.

In [ ]:
grades.plot(y='grade')

Az oszlopdiagramnak több értelme lenne:

In [ ]:
grades.plot(y='grade', kind='bar')

Nem adtuk meg az x tengelyt, ekkor automatikusan a DataFrame indexét használja, ami ebben az esetben a tárgyakat jelenti.

Ábrázolhatnánk félév szerint is egy scatter ploton.

In [ ]:
grades.plot(x='semester', y='grade', kind='scatter')

## Amire érdemes figyelni (GOTCHAs)

### Minden művelet új DataFrame-et ad vissza

Minden művelet egy új DataFrame-mel tér vissza, nem módosítja a paramétereit. Ezt el kell tárolnunk, ha használni akarjuk.

### Egy cellában az utolsó kifejezés visszatérési értéke kiíródik, de a többi nem

Ha a többit is szeretnénk kiírni, használjuk a print függvényt.